# T5 inference用コード

- [学習コード](https://colab.research.google.com/drive/1zDHDNQiEmkuPox_iyzS3XN3mAoc5hU2u?usp=sharing)

- setup.py
  - モデルのパラメータやディレクトリパスの設定
- calcSim.py
  - generate_summary
    - 記事の本文からタイトル(要約文)を生成
  - calc_score
    - rougeスコアの計算
  - make_output
    -  上記2関数をまとめたもの

In [1]:
from setup import *
from calcSim import *

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer


import warnings
warnings.simplefilter('ignore')

/Users/haru/opt/anaconda3/envs/t5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## modelの読み込み、tokenizerの読み込み,スコア計算用オブジェクトの作成

In [2]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True,model_max_length=args.max_input_length)
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

## テスト記事

- https://news.yahoo.co.jp/articles/fc57a5ba61aadead0b4f516ce442308469f6b16d
    - score = 0.4174297833240352
```python
target = "NYダウ、反発"
input = "【ニューヨーク時事】29日のニューヨーク株式相場は、優良株で構成するダウ工業株30種平均が反発。米連邦準備制度理事会（FRB）の金融政策に関心が集まる中、売り買いが交錯し、午前9時35分現在は前日終値比10．96ドル安の3万0936．03ドル。ハイテク株中心のナスダック総合指数は86．55ポイント安の1万1094．99。"
```
- https://news.yahoo.co.jp/articles/fa0d49360ff34408e9ace0af33f73ebdcefe08ea
    - score = 0.4919361714500556
```python
target = "＜独自＞中国人留学生のバイト給与の免税撤廃へ"
input = "日本でアルバイトをする中国人留学生に適用されている給与の免税措置の撤廃に向け、政府が日中租税条約の改正を検討していることが２５日、分かった。給与の免税措置は留学生の交流促進を図る目的で導入されたが、滞在国で課税を受けるという近年の国際標準に合わせる。複数の政府関係者が明らかにした。日中租税条約は１９８３（昭和５８）年に締結された。同条約の２１条では、教育を受けるために日本に滞在する中国人留学生が生計や教育のために得る給与を免税扱いにしている。雇用先の企業を通じて必要な届け出をすれば、生活費や学費に充てるためのアルバイト代は源泉徴収の対象とならず、課税されない。免税措置は、中国に滞在する日本人留学生にも同様に適用される。ただ、日本で働く中国人留学生に比べ、中国でアルバイトを希望する日本人留学生は限られる。また、日本人留学生が中国で就労許可を受けるハードルも高いとされ、中国人留学生が免税を受けるケースの方が圧倒的に多いとみられる。１３日の参院決算委員会では、自民党が「アンバランスが生じている」と指摘した。近年では留学生が受け取るアルバイト給与について、居住する滞在国で課税を受けることが国際標準となっている。このため政府は米国やシンガポール、マレーシアなどとの租税条約を改正する際に、免税規定を削除してきた。一方、中国以外でも韓国やフィリピン、インドネシアなど、免税規定が残る条約もある。政府関係者は「個別の国との接触状況は答えられない」としながらも「関係省庁で連携し、積極的に既存の条約の改正に取り組みたい」と語った。"
```

In [3]:
target = "＜独自＞中国人留学生のバイト給与の免税撤廃へ"
input = "日本でアルバイトをする中国人留学生に適用されている給与の免税措置の撤廃に向け、政府が日中租税条約の改正を検討していることが２５日、分かった。給与の免税措置は留学生の交流促進を図る目的で導入されたが、滞在国で課税を受けるという近年の国際標準に合わせる。複数の政府関係者が明らかにした。日中租税条約は１９８３（昭和５８）年に締結された。同条約の２１条では、教育を受けるために日本に滞在する中国人留学生が生計や教育のために得る給与を免税扱いにしている。雇用先の企業を通じて必要な届け出をすれば、生活費や学費に充てるためのアルバイト代は源泉徴収の対象とならず、課税されない。免税措置は、中国に滞在する日本人留学生にも同様に適用される。ただ、日本で働く中国人留学生に比べ、中国でアルバイトを希望する日本人留学生は限られる。また、日本人留学生が中国で就労許可を受けるハードルも高いとされ、中国人留学生が免税を受けるケースの方が圧倒的に多いとみられる。１３日の参院決算委員会では、自民党が「アンバランスが生じている」と指摘した。近年では留学生が受け取るアルバイト給与について、居住する滞在国で課税を受けることが国際標準となっている。このため政府は米国やシンガポール、マレーシアなどとの租税条約を改正する際に、免税規定を削除してきた。一方、中国以外でも韓国やフィリピン、インドネシアなど、免税規定が残る条約もある。政府関係者は「個別の国との接触状況は答えられない」としながらも「関係省庁で連携し、積極的に既存の条約の改正に取り組みたい」と語った。"

## タイトル生成、類似度計算

In [4]:
cs = CalcSim(trained_model,tokenizer,args.max_input_length,args.max_target_length)
result = cs.make_output(input,target)

政府関係者が明らかにした。日中租税条約の改正を検討する。日本政府関係者が明らかにした。


## 出力

In [5]:
result

0.4919361714500556

In [6]:
import pandas as pd

In [21]:
yahoo = pd.read_csv("yahoo_137.csv")
yahoo_s = yahoo.sort_values('score')

In [22]:
# scores = []
# for i in range(len(yahoo)):
#   print(f"num : {i}")
#   try:
#     output = yahoo.iloc[i]["output"]
#     target = yahoo.iloc[i]["target"]
#     score = cs.make_output(yahoo.iloc[i]["input"],yahoo.iloc[i]["target"])
#     scores.append(score)
#   except:
#     scores.append(-100)


In [23]:
# yahoo_s.to_csv("yahoo_137_sorted.csv")

In [27]:
yahoo_s.iloc[137]["score"]

0.996175516165056

In [28]:
yahoo_s

,input,output,target,score
81,NaN,ông kong : ông kong yuming,NaN,-100.000000
23,は、約3万人の勤務場所を原則として自宅とし、出社は出張にする方針。は去年、従業員の勤務を原則...,新たな勤務制度を導入する。従業員のモチベーションを高める。,「勤務場所は自宅」「飛行機での出社も可」が3万人を対象に来月から新たな勤務制度へ,0.327765
43,llipponewsetwork()19日午後4時すぎ、東京・日野市を流れる多摩川で12歳と...,llipponewsetwork() llipponewsetwork() llippone...,【速報】多摩川で12歳と14歳の少年が流され意識不明,0.366920
109,放送倫理・番組向上機構()放送人権委員会は22日までに、ペットサロンに預けられた犬がシャンプ...,放送倫理・番組向上機構()が22日、ペットサロンの経営者を訴えた。,日テレ「スッキリ」審理へ 人権委員会,0.387114
119,「広島3-3阪神」(23日、マツダスタジアム)広島が延長十回、サヨナラの好機を演出したが、代...,広島3-3阪神(23日、マツダスタジアム),広島 球場騒然 あぁサヨナラ目前で代走・中村奨が大転倒 ホーム手前でタッチアウト,0.425986
...,...,...,...,...
36,テレビ大分大分県大分市の大分シティが運営する温浴施設「シティスパてんくう」でレジオネラ菌が検...,テレビ大分 大分シティ(大分シティ) >シティスパてんくう >レジオネラ菌について テレビ大...,シティスパてんくう レジオネラ菌検出 営業休止に 大分,0.971954
100,【カブール共同】アフガニスタン東部の地震で、死者が千人を超え、負傷者も1500人以上に上った...,アフガニスタンの地震で、死者が1000人以上に上った。,アフガン地震の死者、1000人超に,0.972968
72,韓国の国産宇宙ロケット「ヌリ号」(2)が21日16時、全羅南道・高興の羅老宇宙センターから打...,ヌリ号(2)の打ち上げに成功した。ヌリ号の打ち上げに成功した。,韓国国産宇宙ロケット「ヌリ号」、打ち上げに成功,0.982014
42,プライムオンライン19日午後3時8分ごろに石川・能登地方で発生した地震について、気象庁が午後...,気象庁 プライムオンライン 気象庁 鎌谷紀子氏「揺れが強かった地域では、最大震度6弱の地震に...,気象庁「1週間程度は最大6弱の地震に注意」 石川・能登地方で地震,0.990845
